# Análisis y visualización de datos de Tokyo_Airbnb 

<img src="https://upload.wikimedia.org/wikipedia/commons/6/69/Airbnb_Logo_B%C3%A9lo.svg" style= "height:150px;float:left;">

# FASES

## Fase 1. Entendimiento de datos

# Importar librerias

In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# import geopandas as gpd
import plotly.express as px
from IPython.display import HTML
import plotly.io as pio

# Cargar y leer datos

In [15]:
pd.read_csv('tokyo_airbnb.csv')

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month
0,35303,"La Casa Gaienmae C Harajuku, Omotesando is nearby",151977,Miyuki,NaN,Shibuya Ku,35.67152,139.71203,Private room,4196,28,18,2018-07-28,0.20
1,197677,Oshiage Holiday Apartment,964081,Yoshimi & Marek,NaN,Sumida Ku,35.71721,139.82596,Entire home/apt,10975,3,156,2019-05-21,1.65
2,289597,Private apt in central Tokyo #203,341577,Hide&Kei,NaN,Nerima Ku,35.74267,139.65810,Entire home/apt,4196,30,107,2019-05-25,1.25
3,370759,"Cozy flat #203, local area YET 10 mins to shib...",1573631,"Gilles,Mayumi,Taiki",NaN,Setagaya Ku,35.66344,139.65593,Entire home/apt,6994,29,99,2019-05-07,1.55
4,700253,Private apt in central Tokyo #201,341577,Hide&Kei,NaN,Nerima Ku,35.74264,139.65832,Entire home/apt,3981,30,101,2019-05-03,1.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11461,36083287,1min to station · Skytree/Tatami hotel/new design,235406925,Sayoko佐代子,NaN,Sumida Ku,35.70862,139.81393,Entire home/apt,21951,1,0,NaN,NaN
11462,36083512,Asakusa Wired house 5pax near by UENO/AKIHABARA,267481408,Hyongsu,NaN,Taito Ku,35.72464,139.78078,Entire home/apt,7747,1,0,NaN,NaN
11463,36084566,东京精品民宿,161702799,Yang,NaN,Toshima Ku,35.73587,139.73397,Private room,56275,1,0,NaN,NaN
11464,36085357,#4 DIRECT ACCESS TO DISNEY IN BUS! 3MIN TO STA...,208189463,Shotaro,NaN,Edogawa Ku,35.66255,139.87236,Entire home/apt,6456,1,0,NaN,NaN


### 1.2 Exploración de datos

### 1.3. Verificar calidad de datos

## Fase 2. Preparación de datos

### 2.1. Seleccionar los datos

### 2.2. Limpiar los datos (Amputación e Imputación)

### 2.3. Construcción de los datos

## Fase 3. Visualización de datos

##### etc.....

# Fase 1. Entendimiento de datos

## 1.1. Descripción de los datos

In [16]:
datos = pd.read_csv('tokyo_airbnb.csv')

In [19]:
datos.describe()

,id,host_id,neighbourhood_group,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month
count,1.146600e+04,1.146600e+04,0.0,11466.000000,11466.000000,1.146600e+04,11466.000000,11466.000000,9789.000000
mean,2.640545e+07,1.399776e+08,NaN,35.687159,139.734537,1.498470e+04,3.316239,26.028868,2.262540
std,7.565212e+06,8.064289e+07,NaN,0.217853,0.092771,3.327345e+04,7.766766,38.455449,1.744004
min,3.530300e+04,1.519770e+05,NaN,27.072330,139.130020,0.000000e+00,1.000000,0.000000,0.040000
25%,2.215251e+07,5.783171e+07,NaN,35.679620,139.698975,6.026000e+03,1.000000,2.000000,1.000000
50%,2.841873e+07,1.502326e+08,NaN,35.701910,139.729165,1.000700e+04,1.000000,12.000000,2.060000
75%,3.222416e+07,2.135896e+08,NaN,35.724215,139.784270,1.721600e+04,2.000000,32.000000,3.170000
max,3.608660e+07,2.714130e+08,NaN,35.832430,142.202880,1.000046e+06,180.000000,453.000000,51.210000


In [ ]:
## Si vemos la información aquí mostrada vemos que no se se encuentran todas las columnas de nuestros datos.

In [ ]:
## Con (incluide="all" vemos seguidamente que ya aparecen todas y con más información)

In [29]:
datos.describe(include="all")

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month
count,1.146600e+04,11466,1.146600e+04,11466,11466,11466.000000,11466.000000,11466,1.146600e+04,11466.000000,11466.000000,11466.0,11466.000000
unique,NaN,10810,NaN,2055,56,NaN,NaN,3,NaN,NaN,NaN,414.0,NaN
top,NaN,Near Ueno#Cozy Hotel#6min to Subway#40min airport,NaN,M-1 Tokyo,Shinjuku Ku,NaN,NaN,Entire home/apt,NaN,NaN,NaN,0.0,NaN
freq,NaN,22,NaN,133,1882,NaN,NaN,7463,NaN,NaN,NaN,1677.0,NaN
mean,2.640545e+07,NaN,1.399776e+08,NaN,NaN,35.687159,139.734537,NaN,1.498470e+04,3.316239,26.028868,NaN,1.931624
std,7.565212e+06,NaN,8.064289e+07,NaN,NaN,0.217853,0.092771,NaN,3.327345e+04,7.766766,38.455449,NaN,1.798866
min,3.530300e+04,NaN,1.519770e+05,NaN,NaN,27.072330,139.130020,NaN,0.000000e+00,1.000000,0.000000,NaN,0.000000
25%,2.215251e+07,NaN,5.783171e+07,NaN,NaN,35.679620,139.698975,NaN,6.026000e+03,1.000000,2.000000,NaN,0.530000
50%,2.841873e+07,NaN,1.502326e+08,NaN,NaN,35.701910,139.729165,NaN,1.000700e+04,1.000000,12.000000,NaN,1.720000
75%,3.222416e+07,NaN,2.135896e+08,NaN,NaN,35.724215,139.784270,NaN,1.721600e+04,2.000000,32.000000,NaN,3.000000


## 1.2 Exploración básica del dataset

In [ ]:
datos

In [22]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11466 entries, 0 to 11465
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   11466 non-null  int64  
 1   name                 11466 non-null  object 
 2   host_id              11466 non-null  int64  
 3   host_name            11450 non-null  object 
 4   neighbourhood_group  0 non-null      float64
 5   neighbourhood        11466 non-null  object 
 6   latitude             11466 non-null  float64
 7   longitude            11466 non-null  float64
 8   room_type            11466 non-null  object 
 9   price                11466 non-null  int64  
 10  minimum_nights       11466 non-null  int64  
 11  number_of_reviews    11466 non-null  int64  
 12  last_review          9789 non-null   object 
 13  reviews_per_month    9789 non-null   float64
dtypes: float64(4), int64(5), object(5)
memory usage: 1.2+ MB


#### Descubierto por una simple descripción de los datos:

##### 1. Hay 14 columnas de características en total, cada una con 11466 datos.
##### 2. Hay 4 columnas con algún grado de falta (host_name/neighbourhood_group/last_review/reviews_per_month)

In [23]:
datos['host_name'].fillna(0,inplace = True)

In [24]:
datos['last_review'].fillna(0,inplace = True)

In [25]:
datos['reviews_per_month'].fillna(0,inplace = True)

In [26]:
datos = datos.drop(columns='neighbourhood_group')
datos

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month
0,35303,"La Casa Gaienmae C Harajuku, Omotesando is nearby",151977,Miyuki,Shibuya Ku,35.67152,139.71203,Private room,4196,28,18,2018-07-28,0.20
1,197677,Oshiage Holiday Apartment,964081,Yoshimi & Marek,Sumida Ku,35.71721,139.82596,Entire home/apt,10975,3,156,2019-05-21,1.65
2,289597,Private apt in central Tokyo #203,341577,Hide&Kei,Nerima Ku,35.74267,139.65810,Entire home/apt,4196,30,107,2019-05-25,1.25
3,370759,"Cozy flat #203, local area YET 10 mins to shib...",1573631,"Gilles,Mayumi,Taiki",Setagaya Ku,35.66344,139.65593,Entire home/apt,6994,29,99,2019-05-07,1.55
4,700253,Private apt in central Tokyo #201,341577,Hide&Kei,Nerima Ku,35.74264,139.65832,Entire home/apt,3981,30,101,2019-05-03,1.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11461,36083287,1min to station · Skytree/Tatami hotel/new design,235406925,Sayoko佐代子,Sumida Ku,35.70862,139.81393,Entire home/apt,21951,1,0,0,0.00
11462,36083512,Asakusa Wired house 5pax near by UENO/AKIHABARA,267481408,Hyongsu,Taito Ku,35.72464,139.78078,Entire home/apt,7747,1,0,0,0.00
11463,36084566,东京精品民宿,161702799,Yang,Toshima Ku,35.73587,139.73397,Private room,56275,1,0,0,0.00
11464,36085357,#4 DIRECT ACCESS TO DISNEY IN BUS! 3MIN TO STA...,208189463,Shotaro,Edogawa Ku,35.66255,139.87236,Entire home/apt,6456,1,0,0,0.00


In [ ]:
datos.info()

### Primero, un procesamiento simple y descripción de algunas características.

In [ ]:
datos['neighbourhood'].unique()

In [ ]:
# Hay tres tipos de habitaciones en total: conjunto completo, habitación individual y habitación para varias personas
datos['room_type'].unique()

In [ ]:
#El análisis de las salas muestra que la mayoría de ellas se alquilan en conjunto, y pocas salas públicas se alquilan.
plt.subplot(311)
data = datos['room_type'].value_counts().tolist()
a = datos['room_type'].unique()
plt.bar(x=a, height=data)

In [ ]:
#Se puede ver que host_name está relativamente concentrado, lo que puede confirmar la conclusión extraída en las listas de host_calculadas en el tipo continuo anterior.Vamos a verlo en detalle.
datos[['host_name','name']].groupby('host_name').count().sort_values(by='name', ascending=False)

In [ ]:
#La ubicación de la casa de familia está relativamente concentrada, aquí hay una mirada específica a las áreas
datos['neighbourhood'].value_counts()

In [ ]:
datos['neighbourhood'].value_counts().index[0]

### La mayoría de las casas se concentran en el distrito de 'Shinjuku Ku', 'Taito Ku' y 'Toshima Ku'  .
### El tipo de habitación se basa básicamente en todo el grupo, complementado con una habitación individual, hay menos habitaciones para varias personas

## Explorar Metas
#### 1.Región (distribución, precio)
#### 2.Casas de familia (top 10 casas de familia más populares, cuáles son las características de las casas de familia más populares)
#### 3.Perspectiva del arrendador (clasificación de los tipos de arrendadores, estimaciones de los ingresos de los arrendadores)

In [ ]:
token = 'pk.eyJ1Ijoid2VubGxhIiwiYSI6ImNsNnhmcjFmcjBzbjQzZHFsNXB3YXc0cHAifQ.Yhi8lAIuVUiXA8TltkAMIw'
px.set_mapbox_access_token(token)

fig = px.scatter_mapbox(datos,
                        lat=datos['latitude'],
                        lon=datos['longitude'],
                        color_continuous_scale=px.colors.cyclical.IceFire,
                        size_max=15,
                        zoom=9
                       )
fig.show()
fig.write_html('tokyo_airbnb.html')

In [ ]:
values = datos.neighbourhood.value_counts()
names = datos.neighbourhood.unique().tolist()
fig = px.pie(datos, values=values, names=names, title='Mapa distribución zona B&B')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### Descubre explorando:

#### 1. Más del 60 % de las celebridades se concentran en las tres áreas centrales de Tokyo, el distrito de 'Shibuya Ku', 'Sumida Ku' y 'Nerima Ku'.
#### 2. Para las áreas alrededor del distrito de  'Shibuya Ku', 'Sumida Ku' y 'Nerima Ku', la distribución de casas de familia también está cerca de los límites de estas tres áreas, especialmente 'Setagaya Ku' y 'Arakawa Ku'.
#### 3. Las áreas restantes están distribuidas de manera relativamente uniforme y no hay un centro obvio.

In [ ]:
#A continuación, continuaremos observando si el precio de las habitaciones en diferentes áreas será diferente
#Para evitar la interferencia de algunos valores atípicos, verifiquemos la distribución de precios antes de realizar un procesamiento razonable
sns.histplot(datos['price'],color='b')

In [ ]:
#Consulta los precios en cada región
a = datos[['neighbourhood','price']].groupby(['neighbourhood','price']).count().reset_index()
for i in names:
    plt.hist(a[a['neighbourhood']==i].price)
    plt.xlabel(i)
    plt.show()

In [ ]:
b = pd.DataFrame(datos['neighbourhood'].unique(),columns=['área'])
b['precio alto'] = datos[['price','neighbourhood']].groupby('neighbourhood').max().price.tolist()
b['precio bajo'] = datos[['price','neighbourhood']].groupby('neighbourhood').min().price.tolist()
b['precio mediano'] = datos[['price','neighbourhood']].groupby('neighbourhood').median().price.tolist()
b['25% de precio'] = datos[['price','neighbourhood']].groupby('neighbourhood').quantile(0.25).price.tolist()
b['75% de precio'] = datos[['price','neighbourhood']].groupby('neighbourhood').quantile(0.75).price.tolist()
b['cuartil'] = np.array(datos[['price','neighbourhood']].groupby('neighbourhood').quantile(0.75).price.tolist()) - np.array(datos[['price','neighbourhood']].groupby('neighbourhood').quantile(0.25).price.tolist())
b

In [ ]:
b['precio mediano'].max()

### Del análisis anterior, se puede concluir que
#### 1.El precio de las casas de familia en Shinagawa Ku es generalmente el más alto (89955.0), seguido de Katsushika Ku (50035.0), y el tercero es Hinohara Mura (28460.5), y los precios de estas tres áreas están ampliamente distribuidos.
#### 2.Sumida Ku, Shibuya Ku y Katsushika Ku, que tienen la mayor cantidad de casas de familia, tienen un precio de alrededor de 7.000, lo que explica en cierta medida la razón de la gran cantidad de casas de familia en la región.
#### 3.Hay una gran diferencia entre el precio más alto y el precio más bajo en cada área, y el precio más alto en algunas áreas supera el millón de yuanes. No está claro si es un valor anormal o si el precio de la casa de familia en sí es tan alto. puede analizarse más a fondo en el área de grandes conjuntos de datos.

## Airbnb
### *top10 más populares
### *¿Por qué es el más popular?

In [ ]:
# Aquí, por el momento, creemos que una reseña representa a un huésped, y aquellos con una reseña promedio/revisión mensual superior al 75 % de los dígitos son los B&B más populares.
avg_review = datos['number_of_reviews'].quantile(0.9)
avg_month_review = datos['reviews_per_month'].quantile(0.9)
print(avg_review)
print(avg_month_review)

In [ ]:
popular_house = datos[(datos['number_of_reviews']>avg_review) & (datos['reviews_per_month']>avg_month_review)]
popular_house.sort_values(by=['number_of_reviews','reviews_per_month'],ascending=False).head(10)

### La mayoría de las casas de familia más populares están en Shinjuku Ku (4 casas), algunas están en Katsushika Ku y Katsushika Ku y Taito Ku (2 cada una).
### Las 10 casas de familia más populares son Entire. Se considera temporalmente que si desea hacer casas de familia, el Entire puede ser una buena opción.